In [363]:
import pandas as pd
from datetime import datetime 
import shutil
import os
import sqlite3 as sql

import xlsxwriter

import configparser
import re
import pathlib

In [362]:
pathlib.Path().resolve()


PosixPath('/Users/ryanapfel/Documents/UCLA/box_transfer')

In [350]:
config = configparser.ConfigParser()

In [352]:
config.read('config.cfg')
config.sections()

['destination', 'sourcePaths', 'logs', 'database']

In [62]:
database_name = 'uploads.sql'

In [64]:
con = sql.connect(database_name)
cur = con.cursor()



cur.execute('''CREATE TABLE uploads
            (date text, study text,  root_dir text, file text)''')
cur.execute('''Create Table errors
                (date date, study text, error text)''')
cur.execute('''Create Table test
                (one text, study text, two text)''')
con.commit()

con.close()


In [70]:

def insert_into(db_filename, to_insert):
    with sql.connect(db_filename) as conn:
        cursor = conn.cursor()
        cursor.executemany('INSERT INTO test VALUES(?,?,?);',to_insert)

    show_test(conn)


def show_test(conn):
    cursor = conn.cursor()
    cursor.execute('select * from test')
    for a,b,c in cursor.fetchall():
        print('  ', a,b,c)

records = [('12', 'Glen','fsjk' ),
			('43', 'tit','fff' ),
			('1920', 'Helo','aaa' )]

insert_into(database_name, records)


   12 Glen fsjk
   43 tit fff
   1920 Helo aaa
   12 Glen fsjk
   43 tit fff
   1920 Helo aaa
   12 Glen fsjk
   43 tit fff
   1920 Helo aaa
   12 Glen fsjk
   43 tit fff
   1920 Helo aaa
   12 Glen fsjk
   43 tit fff
   1920 Helo aaa


In [283]:
class FileTransfer:
    def __init__(self, database_path, rootDirectories, destDir, verbose=False,):
        self.verbose = verbose
        self.database_path = database_path
        self.rootDirectories = rootDirectories
        self.destDir = destDir
        self.alreadyUploaded = self.already_transfered()
        self.uploaded = []
        self.errors = []
        self.DEPTH = 1

    def run_query(self, q):
        with sql.connect(self.database_path) as conn:
            cursor = conn.cursor()
            cursor.execute(q)
    def paramterized_query(self, q, params):
         with sql.connect(self.database_path) as conn:
            cursor = conn.cursor()
            cursor.execute(q, params)


    '''
    args: src dest of files
    returns: True if transfer succesful
    '''
    def moveFiles(self, src):
        try:
            shutil.copy(src, self.destDir)  
            return True
        except Exception as e:
            return False


    def addToDB(self, study, success, src, file):
        if success:
            query = f'''INSERT INTO uploads VALUES (?,?,?,?)'''
            params = (datetime.now(),study,src,file)
            self.uploaded.append(params)
        else:
            query = f'''INSERT INTO errors VALUES (?,?,?)'''
            params = (datetime.now(),study,src)
            self.errors.append(params)

        if self.verbose and success:
            print(f'{study} -- moved {file} to destination')
        elif self.verbose and not success:
            print(f'{study} -- ERROR moving {file} to destination')

        
        
        self.paramterized_query(query, params)



    def already_transfered(self):
        with sql.connect(self.database_path) as conn:
            cursor = conn.cursor()
            cursor.execute(f'''SELECT file FROM uploads''')
            sqlOut = cursor.fetchall()
        
        return [item for sublist in sqlOut for item in sublist] 

    def transfer(self, *args):
        studiesMoved = 0

        for study in self.getSearchableStudies(args):
            path = self.rootDirectories[study]
            for root, _, files in self.walklevel(path, self.DEPTH):
                for file in files:
                    src = f'{root}/{file}'
                    if file.endswith('.zip') and file not in self.alreadyUploaded:
                        succesful = self.moveFiles(src)
                        self.addToDB(study, succesful, src, file)
                        studiesMoved += 1
                    elif file.endswith('.pdf') and file not in self.alreadyUploaded:
                        # TODO: store pdf's somewhere as they might be valueable
                        pass
                    elif file in self.alreadyUploaded and self.verbose:
                        print(f'Already added {file}')

    def insert_into(self, to_insert):
        with sql.connect(self.database_path) as conn:
            cursor = conn.cursor()
            cursor.executemany('INSERT INTO uploads VALUES(?,?,?,?);',to_insert)
    
    def reset_db(self, *args):
        if not args:
            query = f'''DELETE FROM uploads'''
            self.run_query(query)
        else:
            for arg in args:
                query = f"""DELETE FROM uploads WHERE study LIKE '%{arg}%'"""
                self.run_query(query)



    def getSearchableStudies(self, args):
        searchStudies = []

        if not args:
            searchStudies = self.rootDirectories.keys()
        else:
            for arg in args:
                if arg in self.rootDirectories:
                    searchStudies.append(arg)
                else:
                    print(f'{arg} not a valid study, please check declaration')
        
        print('Searching : ',' '.join(searchStudies))

        return searchStudies
            
    def walklevel(self, dir, level):
        some_dir = dir.rstrip(os.path.sep)
        assert os.path.isdir(some_dir)
        num_sep = some_dir.count(os.path.sep)
        for root, dirs, files in os.walk(some_dir):
            yield root, dirs, files
            num_sep_this = root.count(os.path.sep)
            if num_sep + level <= num_sep_this:
                del dirs[:]
    

    def fillDataBase(self, *args):   
        insertionList = []
        
        for study in self.getSearchableStudies(args):
            path = self.rootDirectories[study]
            for root, _, files in self.walklevel(path, self.DEPTH):
                for file in files:
                    if file.endswith('.zip') and file not in self.alreadyUploaded:
                        src = f'{root}/{file}'
                        insertionList.append((datetime.now(), study, src, file))
                        self.uploaded.append((True,study,src, file))
                        

        self.insert_into(insertionList)
        print("Finished Inserting")


    
    def create_log(self, log_destination):
        df = pd.DataFrame(self.uploaded, columns=['Date','Study','Path','File'])
        errors = pd.DataFrame(self.uploaded, columns=['Date','Study','File'])

        writer = pd.ExcelWriter(log_destination, engine='xlsxwriter')
        df.to_excel(writer, sheet_name='Uploaded')
        errors.to_excel(writer, sheet_name='Errors')

        worksheet = writer.sheets['Uploaded']
        errorWorksheet = writer.sheets['Errors']
        # Get the dimensions of the dataframe.
        (max_row, max_col) = df.shape
        # Set the column widths, to make the dates clearer.
        worksheet.set_column(0, max_col, 20)
        
        (max_row, max_col) = errors.shape
        errorWorksheet.set_column(0, max_col, 20)

        writer.save()


        

        
        
        
        

In [341]:
destDir

'/Users/ryanapfel/Documents/Horos Data/INCOMING.noindex'

In [284]:

# rootDirectories = {}
rootDirectories = {"PROST":'/Users/ryanapfel/Library/CloudStorage/Box-Box/NEW_PROST/imaging',
                "EXCELLENT":'/Users/ryanapfel/Library/CloudStorage/Box-Box/EXCELLENT/imaging/SHARED',
                "Timeless":'/Users/ryanapfel/Library/CloudStorage/Box-Box/TIMELESS IMaging Shared/kbeasley/20289_TIMELESS_imaging'}

skipDirs = ['leeeyan','avechera-EXCELLENT']

userPath = os.path.expanduser('~')
destDir = f'{userPath}/Documents/Horos Data/INCOMING.noindex'

ft = FileTransfer(database_name, rootDirectories, destDir,skipDirs, verbose=True)

In [339]:
pd.DataFrame(ft.uploaded, columns=['Date','Study','Path','File'])

,Date,Study,Path,File
0,2022-02-26 17:16:16.825480,EXCELLENT,/Users/ryanapfel/Library/CloudStorage/Box-Box/...,00720-005^.[00720-005].zip
1,2022-02-26 17:17:07.110343,EXCELLENT,/Users/ryanapfel/Library/CloudStorage/Box-Box/...,00720-013^.[00720-013] (1).zip
2,2022-02-26 17:17:47.424680,EXCELLENT,/Users/ryanapfel/Library/CloudStorage/Box-Box/...,00720-008^.[00720-008].zip
3,2022-02-26 17:18:09.634884,EXCELLENT,/Users/ryanapfel/Library/CloudStorage/Box-Box/...,00720-002^.[00720-002].zip


In [286]:
ft.transfer("EXCELLENT")

Searching :  EXCELLENT
Already added Temp12-1.zip
	Moving /Users/ryanapfel/Library/CloudStorage/Box-Box/EXCELLENT/imaging/SHARED/Site 720/00720-017^.[00720-017].zip
EXCELLENT -- ERROR moving 00720-017^.[00720-017].zip to /Users/ryanapfel/Documents/Horos Data/INCOMING.noindex
	Moving /Users/ryanapfel/Library/CloudStorage/Box-Box/EXCELLENT/imaging/SHARED/Site 720/00720-005^.[00720-005].zip
EXCELLENT -- moved 00720-005^.[00720-005].zip to /Users/ryanapfel/Documents/Horos Data/INCOMING.noindex
	Moving /Users/ryanapfel/Library/CloudStorage/Box-Box/EXCELLENT/imaging/SHARED/Site 720/00720-013^.[00720-013] (1).zip
EXCELLENT -- moved 00720-013^.[00720-013] (1).zip to /Users/ryanapfel/Documents/Horos Data/INCOMING.noindex
	Moving /Users/ryanapfel/Library/CloudStorage/Box-Box/EXCELLENT/imaging/SHARED/Site 720/00720-008^.[00720-008].zip
EXCELLENT -- moved 00720-008^.[00720-008].zip to /Users/ryanapfel/Documents/Horos Data/INCOMING.noindex
	Moving /Users/ryanapfel/Library/CloudStorage/Box-Box/EXCEL

KeyboardInterrupt: 

# Spreadsheet PlayAround

In [330]:
df = pd.read_csv('spreadsheets/missing.csv')
df2 = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})


In [336]:
writer = pd.ExcelWriter('spreadsheets/new.xlsx', engine='xlsxwriter')

df.to_excel(writer, sheet_name='Tracker')
df2.to_excel(writer, sheet_name='Playground')


workbook  = writer.book
worksheet = writer.sheets['Tracker']
worksheet2 = writer.sheets['Playground']


# Get the dimensions of the dataframe.
(max_row, max_col) = df.shape

# Set the column widths, to make the dates clearer.
worksheet.set_column(1, max_col, 20)
worksheet.set_column(0, 1, 3)
worksheet.autofilter(0, 0, max_row, max_col - 1)

# Apply a conditional format to the required cell range.
worksheet2.conditional_format(f'B2:B{max_row}',{'type': '2_color_scale'})
writer.save()

                

/opt/anaconda3/envs/box-transfer/lib/python3.10/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


# Horos DB Explore



In [9]:
from os import listdir
from os.path import isfile, join

In [66]:
'''
Returns dataframe to do work directly from hors database
'''


status_map = {0:'Not Proccessed',
            1: 'Ready For Review',
            3: 'Issues with Imaging',
            2:'In Review',
            4:'Ajudicated'}


def extract(path):
    horoscon = sql.connect(db_path)
    horoscur = horoscon.cursor()
    query =  """select datetime(ZDATE,'unixepoch','31 years','localtime') as acquistion_time,
            datetime(ZDATEADDED,'unixepoch','31 years','localtime') as date_added, 
            ZNAME as patient_name,
            ZMODALITY as modality,
            ZSTATETEXT as status,
            Z_PK,
            ZCOMMENT
            from ZSTUDY"""

    dfh = pd.read_sql(query, horoscon)
    return dfh



def split_patient_name(row):
    try:
        split_data  = re.split('[-_]', row.patient_name)
        other = []
        for idx, splitItem in enumerate(split_data):
            if idx == 0:
                row['Study'] = splitItem
            elif idx == 1:
                row['Site_id'] = splitItem
            elif idx == 2:
                row['Subject_id'] = splitItem
            elif idx == 3:
                row['Timepoint'] = splitItem
            else:
                other.append(splitItem)
            
            row['Other'] = other
    except:
        pass
    
    return row


def split_date_time(row):
    dt = datetime.strptime(row.acquistion_time,'%Y-%m-%d %H:%M:%S')
    row['aq_date'] = dt.date()
    row['aq_time'] = dt.time()
    return row

def transform(df):
    df = df.apply(split_patient_name, axis=1)
    df = df.apply(split_date_time, axis=1)
    df['Status'] = df.apply(lambda x: status_map[x.status], axis=1)
    return df[['Study','Site_id','Subject_id','Timepoint','aq_date','aq_time','acquistion_time','Status','patient_name','date_added']]


# https://xlsxwriter.readthedocs.io/example_pandas_conditional.html#ex-pandas-conditional
def write_excel(df, file_path):
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Tracker')
    writer.save()


def load(df, file_path):
    write_excel(df, file_path)


In [65]:
df = extract(db_path)
df = transform(df)
load(df, 'spreadsheets/master_tracker.xlsx')

In [15]:
d

['Timeless', '243', '3490', 'aa']

In [8]:
user = os.path.expanduser('~')
db_path = f'{user}/Documents/Horos Data/Database.sql'


In [355]:
user = os.path.expanduser('~')
user

os.getcwd()

'/Users/ryanapfel/Documents/UCLA/box_transfer'

In [18]:
# query =  """select datetime(ZDATE,'unixepoch','31 years','localtime') as acquistion_time,
#             datetime(ZDATEADDED,'unixepoch','31 years','localtime') as date_added, 
#             ZNAME as patient_name,
#             ZMODALITY as modality,
#             Z_PK,
#             ZCOMMENT
#             from ZSTUDY"""

# dfh = pd.read_sql(query, horoscon)
# dfh.head()

In [32]:
fname = f'Prost_out-{datetime.now().date()}.csv'
dff[['Site_id','Subject_id','Timepoint','acquistion_time', 'modality']].to_csv(fname)

In [87]:
sss =str(dfh.iloc[80]['ZDATEADDED']).split('.')[0]
datetime.strptime(sss, '%Y%m%d%H%M%S')
sss

'666322004'

In [8]:
pd.read_sql("SELECT * FROM ZIMAGE", horoscon)

,Z_PK,Z_ENT,Z_OPT,ZFRAMEID,ZGENERATEDBYOSIRIX,ZINSTANCENUMBER,ZPATHNUMBER,ZSTOREDHEIGHT,ZSTOREDINDATABASEFOLDER,ZSTOREDISKEYIMAGE,...,ZCOMMENT,ZCOMMENT2,ZCOMMENT3,ZCOMMENT4,ZPATHSTRING,ZSTATETEXT,ZSTOREDEXTENSION,ZSTOREDFILETYPE,ZSTOREDMODALITY,ZCOMPRESSEDSOPINSTANCEUID
0,66366,2,1,NaN,NaN,2,49859,1.0,None,None,...,None,None,None,None,None,None,None,None,PR,"b';\'\xb9Q\xb2\xb2%&+K+""\x18\xa2\x1b:YA{E\'\xb..."
1,66367,2,1,NaN,NaN,2,49858,1.0,None,None,...,None,None,None,None,None,None,None,None,PR,"b';\'\xb9Q\xb2\xb2%&+K+""\x18\xa2\x1b:YA{E\'\xb..."
2,66368,2,1,NaN,NaN,3,49860,1.0,None,None,...,None,None,None,None,None,None,None,None,PR,"b';\'\xb9Q\xb2\xb2%&+K+""\x18\xa2\x1b:YA{E\'\xb..."
3,66369,2,1,NaN,NaN,1,49857,1.0,None,None,...,None,None,None,None,None,None,None,None,PR,"b';\'\xb9Q\xb2\xb2%&+K+""\x18\xa2\x1b:YA{E\'\xb..."
4,66370,2,1,NaN,NaN,1,49856,1.0,None,None,...,None,None,None,None,None,None,None,None,PR,"b';\'\xb9Q\xb2\xb2%&+K+""\x18\xa2\x1b:YA{E\'\xb..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138769,491086,2,1,NaN,NaN,591,442194,NaN,None,None,...,None,None,None,None,None,None,None,None,None,"b""+K#\xb3\xb2!\x8bk+[x1\x1bA\x11\x112\x1a\x17\..."
138770,491087,2,1,NaN,NaN,426,442029,NaN,None,None,...,None,None,None,None,None,None,None,None,None,b'+K#\xb3\xb2!\x8bk+[x1\x1bA\x11\x112\x1a\x17\...
138771,491088,2,1,NaN,NaN,17,443391,637.0,None,None,...,None,None,None,None,None,None,None,None,None,b'+K#\xb3\xb2!\x8bk+[x1\x1bA\x11\x112\x1a\x17\...
138772,491089,2,1,NaN,NaN,312,441161,1024.0,None,None,...,None,None,None,None,None,None,None,None,XA,b'+K#\xb3\xb2!\x8bk[\x8b3!\xabA\x11\x112\x1a\x...


In [25]:
pd.read_sql("SELECT * FROM ZSTUDY", horoscon).columns

Index(['Z_PK', 'Z_ENT', 'Z_OPT', 'ZEXPANDED', 'ZHASDICOM', 'ZLOCKEDSTUDY',
       'ZNUMBEROFIMAGES', 'ZSTATETEXT', 'ZDATE', 'ZDATEADDED', 'ZDATEOFBIRTH',
       'ZDATEOPENED', 'ZACCESSIONNUMBER', 'ZCOMMENT', 'ZCOMMENT2', 'ZCOMMENT3',
       'ZCOMMENT4', 'ZDICTATEURL', 'ZID', 'ZINSTITUTIONNAME', 'ZMODALITY',
       'ZNAME', 'ZPATIENTID', 'ZPATIENTSEX', 'ZPATIENTUID',
       'ZPERFORMINGPHYSICIAN', 'ZREFERRINGPHYSICIAN', 'ZREPORTURL',
       'ZSTUDYINSTANCEUID', 'ZSTUDYNAME', 'ZWINDOWSSTATE'],
      dtype='object')

In [32]:
pd.read_sql("""SELECT name
              FROM sqlite_schema
               WHERE type ='table' AND name NOT LIKE 'sqlite_%';""", horoscon)

,name
0,ZALBUM
1,Z_1STUDIES
2,ZIMAGE
3,ZLOGENTRY
4,ZSERIES
5,ZSTUDY
6,Z_PRIMARYKEY
7,Z_METADATA
8,Z_MODELCACHE


# PRost

In [21]:
prost_dif = pd.read_csv('prost_diff.csv')
prost_dif.head()

,Subject ID,Subject #
0,DE-22-001,US-02-004
1,DE-22-002,US-02-002
2,DE-22-003,US-02-003
3,DE-22-004,US-02-001
4,DE-22-005,US-02-001


In [10]:
# set(prost_dif['Corelab_Ids'].dropna()).difference(set(prost_dif['Prost_ID'].dropna()))



In [22]:
missinglist = list(set(prost_dif['Subject ID'].dropna()).difference(set(prost_dif['Subject #'].dropna())))
targetList = [missinglist, missinglist, missinglist]

In [23]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list


In [24]:
timepoints = []
for name in ['Screening','Procedure','24 Hour']:
    for i in missinglist:
        timepoints.append(name)


In [25]:
targetList = flatten_list(targetList)

In [26]:
siteList = []
for i in targetList:
    one = re.split('-', i)[0]
    two = re.split('-', i)[1]
    siteList.append(f'{one}-{two}')

In [27]:
pd.DataFrame([siteList , targetList, timepoints]).T.rename(columns={0:'Site',1:'Subject', 2:'Timepoint'}).to_csv('missing.csv')

Sepia

In [378]:
path = '/Users/ryanapfel/Library/CloudStorage/Box-Box/R01 SEPIA/Resources/CFD'

writer = pd.ExcelWriter('spreadsheets/sepia_files.xlsx', engine='xlsxwriter')


df = pd.DataFrame(os.listdir(path), columns=['filename'])

df.to_excel(writer, sheet_name='Files')

writer.save()